In [1]:
import pandas as pd
import numpy as np
import pysentani as sti

In [2]:
survey = pd.read_excel('../data-anonymous/sentani-merged-cleaned-anonymous-2014-11-29.xlsx')

In [3]:
survey['access_type'] = sti.access_type(survey)
survey['electricity_monthly'] = sti.elec_expenditure_monthly(survey)

In [4]:
# looking only at survey data in Ajau
survey = survey[survey['village_name'] == 'Ajau']

In [5]:
# selecting columns to analyze
# access type and village name included for verification
subsurvey = survey[['access_type','village_name','PLN_expenditure_monthly']]

In [6]:
#there appears to be one significant outlier
subsurvey.dropna()

,access_type,village_name,PLN_expenditure_monthly
348,PLN_grid,Ajau,30000
349,PLN_grid,Ajau,80000
350,PLN_grid,Ajau,50000
352,PLN_grid,Ajau,80000
353,PLN_grid,Ajau,20000
354,PLN_grid,Ajau,30000
355,PLN_grid,Ajau,50000
356,PLN_grid,Ajau,50000
358,PLN_grid,Ajau,30000
359,PLN_grid,Ajau,250000


In [7]:
subsurvey.mean()

PLN_expenditure_monthly    59166.666667
dtype: float64

$$expenditure\ in\ rp \left(\frac{1USD}{12,000rp}\right) \left(\frac{1kWh}{0.08USD}\right)=\left(\frac{expenditure}{960}\right)kWh $$

In [8]:
# function definition converting expenditure into kwh
# 12000 rp to 1 usd was the exchange rate at time of survey along with reported .08USD per kwh
def Ajau_kwh_conversion(cv):
    converted = (cv['PLN_expenditure_monthly']/(960))
    return converted

survey['Ajau_kWh'] = survey.apply(Ajau_kwh_conversion, axis=1)

In [9]:
#average expenditure and kwh per household per month
#groupby access type shown to verify grid type
household_mean = survey.groupby(['access_type'])[['PLN_expenditure_monthly','Ajau_kWh']].mean()
household_mean.reset_index(inplace=True)
household_mean

,access_type,PLN_expenditure_monthly,Ajau_kWh
0,PLN_grid,59166.666667,61.631944


In [10]:
# this one liner may be better
survey[['PLN_expenditure_monthly','Ajau_kWh']].mean()

PLN_expenditure_monthly    59166.666667
Ajau_kWh                      61.631944
dtype: float64

Approximately 3 times lower than metered data. The expenditure outlier, if in fact a misreported number, increases the result by a factor of one-half.

The problem with this analysis is it does not really link appliance ownership to energy use. 